In [46]:
import boto3
import importlib
import utils  
importlib.reload(utils)

<module 'utils' from '/Users/i589142/Documents/GitHub/cloud-computing/utils.py'>

In [47]:
REGION = "eu-north-1"  
KEY_NAME = "test-keypair"
MY_PUBLIC_IP = '193.16.224.9/32'
KEY_NAME = "test-keypair"
AMI_UBUNTU = "ami-042b4708b1d05f512"
INSTANCE_TYPE = "t3.micro"

In [49]:
ec2_client = boto3.client("ec2", region_name=REGION)
ec2_resource = boto3.resource("ec2", region_name=REGION)

In [50]:
key_file_path = utils.create_key_pair(ec2_client, key_name=KEY_NAME)

Key-Pair test-keypair wurde erstellt und gespeichert.


In [51]:
print(key_file_path)

test-keypair.pem


In [52]:
security_group_id = utils.create_security_group(ec2_client, group_name="main-security-group", my_public_ip=MY_PUBLIC_IP, desc="My decription.", ssh=True, http=True, flask=True)

Created security group sg-035c10501660ed711
Security Group ingres authorized.


In [53]:
instance = utils.create_instance(ec2_resource, instance_name="test-vm", image_id=AMI_UBUNTU, instance_type=INSTANCE_TYPE, key_name=KEY_NAME, security_group_id=security_group_id)

Instanz-ID: i-0e4a293abba32d0e1 wird gestartet...
Instanz läuft. Öffentliche IP: 16.171.173.80


In [54]:
!chmod 400 {key_file_path}

In [55]:
!ssh-keyscan {instance.public_ip_address} >> ~/.ssh/known_hosts

In [56]:
!ansible-playbook simple_flask_app_in_docker.yml -i {instance.public_ip_address}, -u ubuntu --private-key {key_file_path}


PLAY [Deploy Flask app in Docker] **********************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Platform linux on host 16.171.173.80 is using the discovered Python
interpreter at /usr/bin/python3.12, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
ok: [16.171.173.80]

TASK [Install required packages] ***********************************************
changed: [16.171.173.80]

TASK [Ensure Docker service is running] ****************************************
ok: [16.171.173.80]

TASK [Clone Flask app repository] **********************************************
changed: [16.171.173.80]

TASK [Build Docker image] ******************************************************
changed: [16.171.173.80]

TASK [Run container] *********************************************

In [ ]:
!kubectl apply -f deployment.yaml

In [57]:
instance.terminate()

{'TerminatingInstances': [{'InstanceId': 'i-0e4a293abba32d0e1',
   'CurrentState': {'Code': 32, 'Name': 'shutting-down'},
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'd2838098-852a-48e1-aadc-29d9ddf0848f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2838098-852a-48e1-aadc-29d9ddf0848f',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '426',
   'date': 'Tue, 01 Jul 2025 11:44:41 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [59]:
ec2_client.delete_security_group(GroupId=security_group_id)

{'Return': True,
 'GroupId': 'sg-035c10501660ed711',
 'ResponseMetadata': {'RequestId': 'e1fc88d6-1f28-4906-9992-1b82276eea16',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e1fc88d6-1f28-4906-9992-1b82276eea16',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '266',
   'date': 'Tue, 01 Jul 2025 11:45:21 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [60]:
ec2_client.delete_key_pair(KeyName=KEY_NAME)

{'Return': True,
 'KeyPairId': 'key-0520d2543eade30f2',
 'ResponseMetadata': {'RequestId': 'afcd6831-88ae-4447-981f-4e17cdf932c8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'afcd6831-88ae-4447-981f-4e17cdf932c8',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Tue, 01 Jul 2025 11:45:23 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}